# Import packages

In [ ]:
import pandas as pd
import re

import sys
sys.path.append('../')
from aqc_utils.gaussian_log_extractor import *
from aqc_utils.molecule import molecule
from aqc_utils.slurm_manager import slurm_manager
from aqc_utils.helper_classes import slurm_status

import logging
logging.basicConfig(level=logging.INFO)

# Objectif :
definir une fonction qui permet de generer un dataframe ou d'updater un .csv avec les molecules calculées et toutes leurs caracteristiques

## First Step
Recupérer les datas d'une molécule

In [ ]:
sm=slurm_manager(user='', host='')
sm.connect()

In [ ]:
jobs_c = sm.get_jobs(slurm_status.created)
sm.update_status(sm.get_jobs())
sm.update_submitted_jobs()

In [ ]:
sm.retrieve_jobs()
sm.get_job_stats(split_by_can=True)

In [ ]:
df = sm.get_job_stats(split_by_can=True)
np.sum(df["failed"])
print(np.sum(df["failed"]), np.sum(df["uploaded"]), np.sum(df["done"])), #np.sum(df["incomplete"]))

In [ ]:
sm.squeue()

# It is very important here to update with a suited tag name the molecules !


In [ ]:
sm.upload_done_molecules_to_db(tags="JS_substrate")

In [ ]:
for job in sm.get_jobs(slurm_status.uploaded):
    job_to_change = sm.get_jobs()[job].base_name
    with sm.connection.cd("/home/jschleinitz/gaussian/"):
        try:
            sm.connection.run(f"mv {job_to_change}.chk ../gaussian_uploaded")
            sm.connection.run(f"mv {job_to_change}.out ../gaussian_uploaded")
            sm.connection.run(f"mv {job_to_change}.log ../gaussian_uploaded")
            sm.connection.run(f"mv {job_to_change}.com ../gaussian_uploaded")               
            sm.connection.run(f"mv {job_to_change}.sh ../gaussian_uploaded")
        except:
            pass